# 🦌 Logic Gate Symbol Library 🔣

`IPyElk` can serve as a jumpping off point for building different types of diagrams
based on defined shapes. This notebook demonstrates importing the logic gate symbol
library.

In [ ]:
import json
import pathlib
from dataclasses import asdict

import importnb
import ipywidgets
import networkx as nx
import traitlets
from IPython.display import JSON, SVG

import ipyelk
import ipyelk.diagram.defs.connectors as conn
import ipyelk.diagram.elk_export
import ipyelk.nx
import ipyelk.tools
from ipyelk import Elk
from ipyelk.diagram import layout_options as opt
from ipyelk.diagram.defs.svg import (
    Circle,
    ConnectorDef,
    Def,
    Ellipse,
    Path,
    Point,
    RawSVG,
    Rect,
)
from ipyelk.diagram.elk_model import ElkLabel, ElkNode, ElkPort

# Wiring up ElkDiagram with def library

# Logic Gates

https://upload.wikimedia.org/wikipedia/commons/c/cb/Circuit_elements.svg

In [ ]:
from ipyelk.contrib.library import logic_gates as logic

gate_library = {c.identifier: c.shape for c in logic.Gate.__subclasses__()}

# Simple Graph to draw the Logic Gate Symbol Library

In [ ]:
def logic_library():
    # build graph
    marks = [symbol(id=symbol.__name__) for symbol in logic.Gate.__subclasses__()]
    g = nx.Graph()
    for mark in marks:
        g.add_node(mark.id, **mark.data)

    # configure app
    app = Elk(
        transformer=ipyelk.nx.XELK(layouts={}, source=(g, None)),
        layout={"height": "100%"},
    )
    app.diagram.defs = gate_library
    return app


if __name__ == "__main__":
    app = logic_library()
    display(app)

# 1-bit memory circuit

In [ ]:
def onebit_memory_example():
    # build graph
    sizing_opts = opt.OptionsWidget(
        options=[
            opt.NodeSizeConstraints(),
            opt.NodeLabelPlacement(horizontal="center", vertical="center"),
        ]
    ).value

    gates = [
        logic.Nor_Gate(id="nor 1"),
        logic.Nor_Gate(id="nor 2"),
    ]

    g = nx.MultiDiGraph()
    for node in gates:
        g.add_node(node.id, **node.data)

    g.add_node("set", layoutOptions=sizing_opts)
    g.add_node("reset", layoutOptions=sizing_opts)
    g.add_node("output", layoutOptions=sizing_opts)

    g.add_edge("set", "nor 1", targetPort="b")
    g.add_edge("reset", "nor 2", targetPort="a")
    g.add_edge("nor 1", "nor 2", sourcePort="out", targetPort="b")
    g.add_edge("nor 2", "nor 1", sourcePort="out", targetPort="a")
    g.add_edge("nor 2", "output", sourcePort="out")

    # configure app
    app = Elk(
        transformer=ipyelk.nx.XELK(layouts={}, source=(g, None)),
        layout={"height": "100%"},
        style={" rect.elknode": {"fill": "lightgrey"}},
    )
    app.diagram.defs = gate_library
    return app


if __name__ == "__main__":
    app = onebit_memory_example()
    display(app)